# <center>PROJECT SANDBOX</center>

## Documentation
The aim of this notebook is to provide a simple sandbox to test different NN architectures for the project. , here is a doc about the functions imported from `scripts` folder : 

- **`prepare_dataset(device,ratio=0.5,shuffle_ctx=False)`** :
    - **Input**:
        - device : a torch.device object
        - ratio : a float ratio between 0 and 1 that determines the average proportion of modern english verses in the data loader
        - shuffle_ctx : if `True`, shuffle the contexts within a Batch so that half of the `x_1` elements has a wrong context `ctx_1`. Useful to train the context recognizer model.
    - **Return** :
        - a torch Dataset | class : Shakespeare inherited from torch.utils.data.Dataset
        - a python word dictionary (aka tokenizer) | class : dict
    - **Tensors returned when loaded in the dataloader**:
        - x_1 : input verse (modern / shakespearian)
        - x_2 : output verse (modern / shakespearian)

        - ctx_1 = context of the input verse
        - ctx_2 = context of the output verse

        - len_x : length of the input verse
        - len_y : length of the output verse

        - len_ctx_x : length of the input verse context
        - len_ctx_y : length of the output verse context

        - label : label of the input verse (0 : modern, 1 : shakespearian)
        - label_ctx : label of the context (0 : wrong context, 1 : right context)
- **`string2code(string,dict)`** : 
    - **Input**:
        - string : a sentence
        - dict : a tokenizer
    - **Return** :
        - a torch Longtensor (sentence tokenized)
- **`code2string(torch.Longtensor,dict)`** : 
    - **Input**:
        - torch.Longtensor : a sentence tokenized
        - dict : a tokenizer
    - **Return** :
        - a string sentence

## Importing packages

In [1]:
from scripts.data_builders.prepare_dataset import prepare_dataset,string2code,code2string,assemble


import torch
import torchvision.datasets as datasets
import torch.nn.functional as F
from torch import nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter
import math
from torch.nn import BCELoss,CrossEntropyLoss
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import pickle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device = ",device)

device =  cpu


## Preprocessing data

In [2]:
train_data, dict_words = prepare_dataset(device,ratio=0.5,shuffle_ctx=True) #check with shift+tab to look at the data structure
batch_size = 32
dict_token = {b:a for a,b in dict_words.items()} #dict for code2string

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                           shuffle=True,collate_fn=train_data.collate)
dict_size = len(dict_words)
d_embedding = 300 #cf. paper Y.Kim 2014 Convolutional Neural Networks for Sentence Classification

print("- dict size : ",dict_size)

Loading ...
- Shakespeare dataset length :  21079
- Corrupted samples (ignored) :  0
- dict size :  17513


## Designing NN model

# Model 1 : CNN

In [3]:
class CoherenceClassifier(torch.nn.Module):
    def __init__(self,dict_size=dict_size,d_embedding=300):
        super().__init__()
        self.embed_layer=torch.nn.Embedding(dict_size+1,d_embedding,padding_idx=dict_size)

        self.conv_1 = torch.nn.Conv1d(d_embedding,3,kernel_size = 3, stride = 1)
        self.max_pool = torch.nn.MaxPool1d(3,2)
        self.relu = torch.nn.ReLU()
        self.linear = torch.nn.Linear(3,1)
    
    def forward(self,x):
        x = self.embed_layer(x)
        x = self.conv_1( x.transpose(1,2))
        x = self.max_pool( x )
        x = self.relu( x )
        x = torch.max( x , 2 )[0]
        x = torch.sigmoid(self.linear(x))
        return x

In [4]:
epochs = 100
model = CoherenceClassifier().to(device)
optimizer = optim.Adam(params=model.parameters(),lr=0.01)
loss_func = BCELoss()
n = len(train_data.x) // batch_size

In [5]:
for epoch in range(epochs):
    total_loss = 0
    i = 0
    for _,_ , ctx,_ , _,_ , len_ctx,_, _,label_ctx in train_loader:
        i+=1
        optimizer.zero_grad()
        
        #CNN
        ctx = model.forward(ctx).reshape(-1) #CNN architecture
        
        loss = loss_func( ctx , label_ctx.float() )
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
        #Vizualization

    print('-' * 35)
    print('| epoch {:3d} | '
          'lr {:02.2f} | '
          'loss {:5.2f}'.format(
            epoch+1, optimizer.state_dict()["param_groups"][0]["lr"],
            round(total_loss,2)))

KeyboardInterrupt: 

# Model 2 : LSTM 

In [6]:
class CoherenceClassifier(torch.nn.Module):
    def __init__(self,dict_size=dict_size,d_embedding=300,d_hidden=100):
        super().__init__()
        self.d_hidden = d_hidden
        self.embedding = nn.Embedding(dict_size+1,d_embedding,padding_idx=dict_size)
        self.lstm = nn.LSTM(d_embedding,self.d_hidden,dropout=0.,num_layers=1,bidirectional=False)
        #self.bn0 = nn.BatchNorm1d(self.d_hidden)
        self.linear1 = torch.nn.Linear(self.d_hidden,1)
    
    def forward(self,x,len_x):
        x = self.embedding(x)
        x = pack_padded_sequence(x.permute(1,0,2),len_x,enforce_sorted=False)
        _,x = self.lstm(x)
        x = x[0].reshape(-1,self.d_hidden)
        #x = self.bn0(x)
        x = torch.sigmoid( self.linear1(x) ).reshape(-1)
        return x

In [ ]:
epochs = 100
model = CoherenceClassifier().to(device)
optimizer = optim.Adam(params=model.parameters(),lr=0.01)
loss_func = BCELoss()
n = len(train_data.x) // batch_size

In [ ]:
for epoch in range(epochs):
    total_loss = 0
    i = 0
    for _,_ , ctx,_ , _,_ , len_ctx,_, _,label_ctx in train_loader:
        i+=1
        optimizer.zero_grad()
        
        #LSTM
        ctx = model.forward(ctx,len_ctx) #LSTM architecture
        
        loss = loss_func( ctx , label_ctx.float() )
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    print('-' * 35)
    print('| epoch {:3d} | '
          'lr {:02.2f} | '
          'loss {:5.2f}'.format(
            epoch+1, optimizer.state_dict()["param_groups"][0]["lr"],
            round(total_loss,2)))

# Model 3 : Transformers

In [9]:
class CoherenceClassifier(nn.Module):

    def __init__(self,dict_size=dict_size, d_embedding=300,  dropout=0.1):
        super(CoherenceClassifier, self).__init__()
        
        self.embedding = nn.Embedding(dict_size+1,d_embedding,padding_idx=dict_size)
        self.pos_encoder = PositionalEncoding(d_embedding, dropout)
        encoder_layers = TransformerEncoderLayer(d_model=d_embedding, nhead = 4,dropout=dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, num_layers=4)
        
        self.decoder = nn.Linear(d_embedding, 2 )

    def forward(self, x):
        print("---DIMENSIONS---")
        print(x.shape)
        x = self.embedding( x )
        print(x.shape)
        #x = self.pos_encoder( x )
        x = self.transformer_encoder( x )
        print(x.shape)
        x = torch.softmax(torch.tanh(self.decoder( x )),1)
        return x
    
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=50):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x * math.sqrt(self.d_model)
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [10]:
epochs = 100
model = CoherenceClassifier().to(device)
optimizer = optim.Adam(params=model.parameters(),lr=0.01)
loss_func = CrossEntropyLoss()
n = len(train_data.x) // batch_size

In [ ]:
for epoch in range(epochs):
    total_loss = 0
    i = 0
    for _,_ , ctx,_ , _,_ , len_ctx,_, _,label_ctx in train_loader:
        i+=1
        optimizer.zero_grad()
        
        #Transformer
        ctx = model.forward(ctx) #Transformer architecture
        y = torch.cat([label_ctx.reshape(-1,1),1-label_ctx.reshape(-1,1)],dim=1) # Transformer architecture
        
        loss = loss_func( ctx , y )
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    print('-' * 35)
    print('| epoch {:3d} | '
          'lr {:02.2f} | '
          'loss {:5.2f}'.format(
            epoch+1, optimizer.state_dict()["param_groups"][0]["lr"],
            round(total_loss,2)))

---DIMENSIONS---
torch.Size([32, 106])
torch.Size([32, 106, 300])
torch.Size([32, 106, 300])
---DIMENSIONS---
torch.Size([32, 101])
torch.Size([32, 101, 300])
torch.Size([32, 101, 300])
---DIMENSIONS---
torch.Size([32, 66])
torch.Size([32, 66, 300])
torch.Size([32, 66, 300])
---DIMENSIONS---
torch.Size([32, 86])
torch.Size([32, 86, 300])
torch.Size([32, 86, 300])
---DIMENSIONS---
torch.Size([32, 76])
torch.Size([32, 76, 300])
torch.Size([32, 76, 300])
---DIMENSIONS---
torch.Size([32, 57])
torch.Size([32, 57, 300])
torch.Size([32, 57, 300])
---DIMENSIONS---
torch.Size([32, 92])
torch.Size([32, 92, 300])
torch.Size([32, 92, 300])
---DIMENSIONS---
torch.Size([32, 71])
torch.Size([32, 71, 300])
torch.Size([32, 71, 300])
---DIMENSIONS---
torch.Size([32, 94])
torch.Size([32, 94, 300])
torch.Size([32, 94, 300])
---DIMENSIONS---
torch.Size([32, 92])
torch.Size([32, 92, 300])
torch.Size([32, 92, 300])
---DIMENSIONS---
torch.Size([32, 69])
torch.Size([32, 69, 300])
torch.Size([32, 69, 300])
---D

---DIMENSIONS---
torch.Size([32, 96])
torch.Size([32, 96, 300])
torch.Size([32, 96, 300])
---DIMENSIONS---
torch.Size([32, 78])
torch.Size([32, 78, 300])
torch.Size([32, 78, 300])
---DIMENSIONS---
torch.Size([32, 85])
torch.Size([32, 85, 300])
torch.Size([32, 85, 300])
---DIMENSIONS---
torch.Size([32, 78])
torch.Size([32, 78, 300])
torch.Size([32, 78, 300])
---DIMENSIONS---
torch.Size([32, 71])
torch.Size([32, 71, 300])
torch.Size([32, 71, 300])
---DIMENSIONS---
torch.Size([32, 61])
torch.Size([32, 61, 300])
torch.Size([32, 61, 300])
---DIMENSIONS---
torch.Size([32, 80])
torch.Size([32, 80, 300])
torch.Size([32, 80, 300])
---DIMENSIONS---
torch.Size([32, 103])
torch.Size([32, 103, 300])
torch.Size([32, 103, 300])
---DIMENSIONS---
torch.Size([32, 110])
torch.Size([32, 110, 300])
torch.Size([32, 110, 300])
---DIMENSIONS---
torch.Size([32, 79])
torch.Size([32, 79, 300])
torch.Size([32, 79, 300])
---DIMENSIONS---
torch.Size([32, 56])
torch.Size([32, 56, 300])
torch.Size([32, 56, 300])
---D

---DIMENSIONS---
torch.Size([32, 83])
torch.Size([32, 83, 300])
torch.Size([32, 83, 300])
---DIMENSIONS---
torch.Size([32, 89])
torch.Size([32, 89, 300])
torch.Size([32, 89, 300])
---DIMENSIONS---
torch.Size([32, 51])
torch.Size([32, 51, 300])
torch.Size([32, 51, 300])
---DIMENSIONS---
torch.Size([32, 71])
torch.Size([32, 71, 300])
torch.Size([32, 71, 300])
---DIMENSIONS---
torch.Size([32, 103])
torch.Size([32, 103, 300])
torch.Size([32, 103, 300])
---DIMENSIONS---
torch.Size([32, 77])
torch.Size([32, 77, 300])
torch.Size([32, 77, 300])
---DIMENSIONS---
torch.Size([32, 81])
torch.Size([32, 81, 300])
torch.Size([32, 81, 300])
---DIMENSIONS---
torch.Size([32, 79])
torch.Size([32, 79, 300])
torch.Size([32, 79, 300])
---DIMENSIONS---
torch.Size([32, 97])
torch.Size([32, 97, 300])
torch.Size([32, 97, 300])
---DIMENSIONS---
torch.Size([32, 69])
torch.Size([32, 69, 300])
torch.Size([32, 69, 300])
---DIMENSIONS---
torch.Size([32, 59])
torch.Size([32, 59, 300])
torch.Size([32, 59, 300])
---DIME

---DIMENSIONS---
torch.Size([32, 110])
torch.Size([32, 110, 300])
torch.Size([32, 110, 300])
---DIMENSIONS---
torch.Size([32, 85])
torch.Size([32, 85, 300])
torch.Size([32, 85, 300])
---DIMENSIONS---
torch.Size([32, 108])
torch.Size([32, 108, 300])
torch.Size([32, 108, 300])
---DIMENSIONS---
torch.Size([32, 128])
torch.Size([32, 128, 300])
torch.Size([32, 128, 300])
---DIMENSIONS---
torch.Size([32, 58])
torch.Size([32, 58, 300])
torch.Size([32, 58, 300])
---DIMENSIONS---
torch.Size([32, 93])
torch.Size([32, 93, 300])
torch.Size([32, 93, 300])
---DIMENSIONS---
torch.Size([32, 74])
torch.Size([32, 74, 300])
torch.Size([32, 74, 300])
---DIMENSIONS---
torch.Size([32, 94])
torch.Size([32, 94, 300])
torch.Size([32, 94, 300])
---DIMENSIONS---
torch.Size([32, 77])
torch.Size([32, 77, 300])
torch.Size([32, 77, 300])
---DIMENSIONS---
torch.Size([32, 65])
torch.Size([32, 65, 300])
torch.Size([32, 65, 300])
---DIMENSIONS---
torch.Size([32, 86])
torch.Size([32, 86, 300])
torch.Size([32, 86, 300])
-

torch.Size([32, 57, 300])
---DIMENSIONS---
torch.Size([32, 76])
torch.Size([32, 76, 300])
torch.Size([32, 76, 300])
---DIMENSIONS---
torch.Size([32, 68])
torch.Size([32, 68, 300])
torch.Size([32, 68, 300])
---DIMENSIONS---
torch.Size([32, 57])
torch.Size([32, 57, 300])
torch.Size([32, 57, 300])
---DIMENSIONS---
torch.Size([32, 60])
torch.Size([32, 60, 300])
torch.Size([32, 60, 300])
---DIMENSIONS---
torch.Size([32, 73])
torch.Size([32, 73, 300])
torch.Size([32, 73, 300])
---DIMENSIONS---
torch.Size([32, 68])
torch.Size([32, 68, 300])
torch.Size([32, 68, 300])
---DIMENSIONS---
torch.Size([32, 62])
torch.Size([32, 62, 300])
torch.Size([32, 62, 300])
---DIMENSIONS---
torch.Size([32, 94])
torch.Size([32, 94, 300])
torch.Size([32, 94, 300])
---DIMENSIONS---
torch.Size([32, 62])
torch.Size([32, 62, 300])
torch.Size([32, 62, 300])
---DIMENSIONS---
torch.Size([32, 81])
torch.Size([32, 81, 300])
torch.Size([32, 81, 300])
---DIMENSIONS---
torch.Size([32, 57])
torch.Size([32, 57, 300])
torch.Size

...

---DIMENSIONS---
torch.Size([32, 87])
torch.Size([32, 87, 300])
torch.Size([32, 87, 300])
---DIMENSIONS---
torch.Size([32, 78])
torch.Size([32, 78, 300])
torch.Size([32, 78, 300])
---DIMENSIONS---
torch.Size([32, 87])
torch.Size([32, 87, 300])
torch.Size([32, 87, 300])
---DIMENSIONS---
torch.Size([32, 78])
torch.Size([32, 78, 300])
torch.Size([32, 78, 300])
---DIMENSIONS---
torch.Size([32, 83])
torch.Size([32, 83, 300])
torch.Size([32, 83, 300])
---DIMENSIONS---
torch.Size([32, 96])
torch.Size([32, 96, 300])
torch.Size([32, 96, 300])
---DIMENSIONS---
torch.Size([32, 71])
torch.Size([32, 71, 300])
torch.Size([32, 71, 300])
---DIMENSIONS---
torch.Size([32, 70])
torch.Size([32, 70, 300])
torch.Size([32, 70, 300])
---DIMENSIONS---
torch.Size([32, 68])
torch.Size([32, 68, 300])
torch.Size([32, 68, 300])
---DIMENSIONS---
torch.Size([32, 77])
torch.Size([32, 77, 300])
torch.Size([32, 77, 300])
---DIMENSIONS---
torch.Size([32, 58])
torch.Size([32, 58, 300])
torch.Size([32, 58, 300])
---DIMENSI

---DIMENSIONS---
torch.Size([32, 69])
torch.Size([32, 69, 300])
torch.Size([32, 69, 300])
---DIMENSIONS---
torch.Size([32, 64])
torch.Size([32, 64, 300])
torch.Size([32, 64, 300])
---DIMENSIONS---
torch.Size([32, 55])
torch.Size([32, 55, 300])
torch.Size([32, 55, 300])
---DIMENSIONS---
torch.Size([32, 66])
torch.Size([32, 66, 300])
torch.Size([32, 66, 300])
---DIMENSIONS---
torch.Size([32, 93])
torch.Size([32, 93, 300])
torch.Size([32, 93, 300])
---DIMENSIONS---
torch.Size([32, 64])
torch.Size([32, 64, 300])
torch.Size([32, 64, 300])
---DIMENSIONS---
torch.Size([32, 115])
torch.Size([32, 115, 300])
torch.Size([32, 115, 300])
---DIMENSIONS---
torch.Size([32, 115])
torch.Size([32, 115, 300])
torch.Size([32, 115, 300])
---DIMENSIONS---
torch.Size([32, 58])
torch.Size([32, 58, 300])
torch.Size([32, 58, 300])
---DIMENSIONS---
torch.Size([32, 65])
torch.Size([32, 65, 300])
torch.Size([32, 65, 300])
---DIMENSIONS---
torch.Size([32, 163])
torch.Size([32, 163, 300])
torch.Size([32, 163, 300])
-

torch.Size([32, 80, 300])
---DIMENSIONS---
torch.Size([32, 94])
torch.Size([32, 94, 300])
torch.Size([32, 94, 300])
---DIMENSIONS---
torch.Size([32, 91])
torch.Size([32, 91, 300])
torch.Size([32, 91, 300])
---DIMENSIONS---
torch.Size([32, 67])
torch.Size([32, 67, 300])
torch.Size([32, 67, 300])
---DIMENSIONS---
torch.Size([32, 128])
torch.Size([32, 128, 300])
torch.Size([32, 128, 300])
---DIMENSIONS---
torch.Size([32, 81])
torch.Size([32, 81, 300])
torch.Size([32, 81, 300])
---DIMENSIONS---
torch.Size([32, 63])
torch.Size([32, 63, 300])
torch.Size([32, 63, 300])
---DIMENSIONS---
torch.Size([32, 54])
torch.Size([32, 54, 300])
torch.Size([32, 54, 300])
---DIMENSIONS---
torch.Size([32, 92])
torch.Size([32, 92, 300])
torch.Size([32, 92, 300])
---DIMENSIONS---
torch.Size([32, 59])
torch.Size([32, 59, 300])
torch.Size([32, 59, 300])
---DIMENSIONS---
torch.Size([32, 70])
torch.Size([32, 70, 300])
torch.Size([32, 70, 300])
---DIMENSIONS---
torch.Size([32, 69])
torch.Size([32, 69, 300])
torch.S

---DIMENSIONS---
torch.Size([32, 62])
torch.Size([32, 62, 300])
torch.Size([32, 62, 300])
---DIMENSIONS---
torch.Size([32, 87])
torch.Size([32, 87, 300])
torch.Size([32, 87, 300])
---DIMENSIONS---
torch.Size([32, 94])
torch.Size([32, 94, 300])
torch.Size([32, 94, 300])
---DIMENSIONS---
torch.Size([32, 113])
torch.Size([32, 113, 300])
torch.Size([32, 113, 300])
---DIMENSIONS---
torch.Size([32, 86])
torch.Size([32, 86, 300])
torch.Size([32, 86, 300])
---DIMENSIONS---
torch.Size([32, 52])
torch.Size([32, 52, 300])
torch.Size([32, 52, 300])
---DIMENSIONS---
torch.Size([32, 61])
torch.Size([32, 61, 300])
torch.Size([32, 61, 300])
---DIMENSIONS---
torch.Size([32, 85])
torch.Size([32, 85, 300])
torch.Size([32, 85, 300])
---DIMENSIONS---
torch.Size([32, 58])
torch.Size([32, 58, 300])
torch.Size([32, 58, 300])
---DIMENSIONS---
torch.Size([32, 81])
torch.Size([32, 81, 300])
torch.Size([32, 81, 300])
---DIMENSIONS---
torch.Size([32, 62])
torch.Size([32, 62, 300])
torch.Size([32, 62, 300])
---DIME

---DIMENSIONS---
torch.Size([32, 85])
torch.Size([32, 85, 300])
torch.Size([32, 85, 300])
---DIMENSIONS---
torch.Size([32, 64])
torch.Size([32, 64, 300])
torch.Size([32, 64, 300])
---DIMENSIONS---
torch.Size([32, 74])
torch.Size([32, 74, 300])
torch.Size([32, 74, 300])
---DIMENSIONS---
torch.Size([32, 73])
torch.Size([32, 73, 300])
torch.Size([32, 73, 300])
---DIMENSIONS---
torch.Size([32, 75])
torch.Size([32, 75, 300])
torch.Size([32, 75, 300])
---DIMENSIONS---
torch.Size([32, 79])
torch.Size([32, 79, 300])
torch.Size([32, 79, 300])
---DIMENSIONS---
torch.Size([32, 62])
torch.Size([32, 62, 300])
torch.Size([32, 62, 300])
---DIMENSIONS---
torch.Size([32, 62])
torch.Size([32, 62, 300])
torch.Size([32, 62, 300])
---DIMENSIONS---
torch.Size([32, 135])
torch.Size([32, 135, 300])
torch.Size([32, 135, 300])
---DIMENSIONS---
torch.Size([32, 85])
torch.Size([32, 85, 300])
torch.Size([32, 85, 300])
---DIMENSIONS---
torch.Size([32, 79])
torch.Size([32, 79, 300])
torch.Size([32, 79, 300])
---DIME

# Style classifier 

In [70]:
dict_size=len(dict_words)
print(dict_size)

17513


In [66]:
from scripts.StyleClassifier import *

In [69]:
test()

yo


In [13]:
import ipdb

In [14]:
epochs = 100
style_classifier = StyleClassifier(dict_size=dict_size,d_embedding=300,d_hidden=10).to(device)
optimizer = optim.Adam(params=style_classifier.parameters(),lr=0.01)
loss_func =lambda x: max(BCELoss(x[0].float(),x[2].float()),BCELoss(x[1].float(),x[3].float()))
n = len(train_data.x) // batch_size

In [15]:
for epoch in range(epochs):
    total_loss = 0
    i = 0
    for x,y,_,_ , _,_ ,_,_, label,_ in train_loader:
#         ipdb.set_trace()
        i+=1
        optimizer.zero_grad()
        xim,yim=style_classifier.forward(x,y)
        loss=loss_func([xim,yim,label,1-label])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
#     print('-' * 35)
#     print('| epoch {:3d} | '
#           'lr {:02.2f} | '
#           'loss {:5.2f}'.format(
#             epoch+1, optimizer.state_dict()["param_groups"][0]["lr"],
#             round(total_loss,2)))

RuntimeError: Given groups=1, weight of size 10 300 3, expected input[32, 35, 300] to have 300 channels, but got 35 channels instead

# Pre-trained model

In [ ]:
model = torch.hub.load('huggingface/pytorch-transformers', 'modelForSequenceClassification', 'bert-base-uncased')    # Download model and configuration from S3 and cache.
model = torch.hub.load('huggingface/pytorch-transformers', 'modelForSequenceClassification', './test/bert_model/')  # E.g. model was saved using `save_pretrained('./test/saved_model/')`
model = torch.hub.load('huggingface/pytorch-transformers', 'modelForSequenceClassification', 'bert-base-uncased', output_attention=True)  # Update configuration during loading
assert model.config.output_attention == True
# Loading from a TF checkpoint file instead of a PyTorch model (slower)
config = AutoConfig.from_json_file('./tf_model/bert_tf_model_config.json')
model = torch.hub.load('huggingface/pytorch-transformers', 'modelForSequenceClassification', './tf_model/bert_tf_checkpoint.ckpt.index', from_tf=True, config=config)

# Draft

In [ ]:
for x,y , ctx_x,ctx_y , len_x,len_y , len_ctx_x,len_ctx_y, label,label_ctx in train_loader:
    
    for i in range(x.shape[0]):
        print("\n- x :")
        print(code2string(x[i],dict_token))
        print("- context of x :")
        print(code2string(ctx_x[i],dict_token))
        print("- context label :",label_ctx[i].item())

        print("- len_ctx_x :")
        print(ctx_x)
#         ipdb.set_trace()
    break